In [1]:
# _*_ coding: utf-8 _*_

import os
import sys
import torch
from torch.nn import functional as F
import numpy as np
from torchtext import data
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe

In [2]:
import itertools
from functools import reduce
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [3]:
d_listings=pd.read_csv("../data/airbnb_barcellona/detailed_listings.csv")
pd.options.display.max_colwidth = 50
pd.options.display.max_columns = None

/Users/stella-yj/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (43,61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
from langdetect import detect
def tryDetect(value):
    try:
        ret=detect(value)
    except:
        ret= '-'
    return ret

In [60]:
def top_mid_low(value):
    if value >90:
        return 'top'
    elif value > 80:
        return 'mid'
    else:
        return 'low'

In [115]:
def top_low(value):
    if value >90:
        return 'top'
    else:
        return 'low'

In [7]:
d_listings['summary_ln']=d_listings['summary'].apply(lambda x: tryDetect(x))

In [61]:
d_listings['top_mid_low']=d_listings['review_scores_rating'].apply(lambda x: top_mid_low(x))

In [116]:
d_listings['top_low']=d_listings['review_scores_rating'].apply(lambda x: top_low(x))

In [117]:
en_listings=d_listings[d_listings['summary_ln'] =='en']

In [118]:
summary_non_zero=en_listings[en_listings['number_of_reviews']>=5]

In [119]:
summary_zero=en_listings[en_listings['number_of_reviews']<5]

In [120]:
summary_non_zero.shape

(8617, 109)

In [121]:
summary_zero.shape

(4958, 109)

In [123]:
summary_non_zero[['summary','top_low']][:100].to_csv(
    "summary_non_zero_tl_test.tsv", sep="\t", header=False, index=False)

In [208]:
summary_non_zero[['summary','top_low']][:-100].shape

(8517, 2)

In [122]:
summary_non_zero[['summary','top_low']][:-100].to_csv(
    "summary_non_zero_tl_train.tsv", sep='\t', header=False, index=False)

In [40]:
summary_non_zero[['summary','review_scores_rating']][:100].to_csv("summary_non_zero_test.tsv", sep='\t', header=False, index=False)

In [124]:
summary_non_zero.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,summary_ln,top_mid_low,top_low
1,18674,https://www.airbnb.com/rooms/18674,20191109094909,2019-11-09,Huge flat for 8 people close to Sagrada Familia,110m2 apartment to rent in Barcelona. Located ...,Apartment with 110 m2 located in the 6th floor...,110m2 apartment to rent in Barcelona. Located ...,none,Apartment in Barcelona located in the heart of...,NaN,"Good transport connection, 150 m from metro “...",Free Wifi - air conditioning. We will provide ...,We can provide you all kind of entrance and ti...,Tourist tax at arrival: 2.48 Eur/person/night ...,NaN,NaN,https://a0.muscache.com/im/pictures/13031453/4...,NaN,71615,https://www.airbnb.com/users/show/71615,Mireia And Maria,2010-01-19,"Barcelona, Cataluña, Spain","We are Mireia (39) & Maria (41), two multiling...",within an hour,99%,NaN,f,https://a0.muscache.com/im/users/71615/profile...,https://a0.muscache.com/im/users/71615/profile...,El Camp de l'Arpa del Clot,46.0,46.0,"['email', 'phone', 'reviews', 'jumio', 'govern...",t,t,"Barcelona, CT, Spain",Eixample,la Sagrada Família,Eixample,Barcelona,CT,08025,Barcelona,"Barcelona, Spain",ES,Spain,41.40420,2.17306,t,Apartment,Entire home/apt,8,2.0,3.0,6.0,Real Bed,"{TV,Internet,Wifi,""Air conditioning"",""Wheelcha...",NaN,$60.00,NaN,NaN,$150.00,$50.00,2,$30.00,1,1125,2,7,1125,1125,3.9,1125.0,today,t,15,24,50,312,2019-11-09,20,15,2013-05-27,2019-10-19,87.0,9.0,9.0,10.0,10.0,9.0,9.0,t,HUTB-002062,NaN,t,f,strict_14_with_grace_period,f,f,30,30,0,0,0.25,en,mid,low
2,23197,https://www.airbnb.com/rooms/23197,20191109094909,2019-11-09,FORUM DELUXE 5 MINS WALK CCIB CENTER & SEA!,I do not accept groups of young people under 2...,Elegant spacious apartment suitable for 6. Amp...,I do not accept groups of young people under 2...,none,Strategically located in the area of Parc del ...,"As a Superhost, I aim to make your stay as com...",Tram stop Line T4 Campus Diagonal- Besos just...,You book the entire apartment for yourselves.,"Please do contact Tony, your check-in manager,...",I do not accept groups of young people under 2...,NaN,NaN,https://a0.muscache.com/im/pictures/738532/806...,NaN,90417,https://www.airbnb.com/users/show/90417,Etain (Marnie),2010-03-09,"Catalonia, Spain","Hi. I'm Etain, though everyone calls me by my...",within an hou

In [4]:
tokenize = lambda x: x.split()
TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True, include_lengths=True
                  , batch_first=True, fix_length=500)
LABEL = data.LabelField(dtype=float)
train_data, test_data = data.TabularDataset.splits(path='.'
                                        , train='summary_non_zero_tl_train.tsv'
                                        , test='summary_non_zero_tl_test.tsv'
                                        , format='tsv'
                                        , fields=[('text', TEXT)
                                        , ('tml', LABEL)])

In [210]:
train_data

In [5]:
TEXT.build_vocab(train_data, vectors=GloVe(name='6B', dim=300), min_freq = 2)
LABEL.build_vocab(train_data)

In [6]:
word_embeddings = TEXT.vocab.vectors
print ("Length of Text Vocabulary: " + str(len(TEXT.vocab)))
print ("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
print ("Label Length: " + str(len(LABEL.vocab)))


Length of Text Vocabulary: 10127
Vector size of Text Vocabulary:  torch.Size([10127, 300])
Label Length: 2


In [7]:
train_data, valid_data = train_data.split()

In [8]:
valid_data

In [9]:
train_iter, valid_iter, test_iter = data.BucketIterator.splits((train_data, valid_data, test_data)
                                                               , batch_size=32
                                                               , sort_key=lambda x: len(x.text)
                                                               , repeat=False
                                                               , shuffle=True)

In [10]:
for idx, batch in enumerate(valid_iter):
    print(batch)


[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of si

In [11]:
for idx, batch in enumerate(test_iter):
    print(batch)


[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 4]
	[.text]:('[torch.LongTensor of size 4x500]', '[torch.LongTensor of size 4]')
	[.tml]:[torch.DoubleTensor of size 4]


In [12]:
for idx, batch in enumerate(train_iter):
    print(batch)


[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of si

	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of siz

	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of size 32]

[torchtext.data.batch.Batch of size 32]
	[.text]:('[torch.LongTensor of size 32x500]', '[torch.LongTensor of size 32]')
	[.tml]:[torch.DoubleTensor of siz

In [13]:
vocab_size = len(TEXT.vocab)

## 이상 data 완료------------------------------------------

## 이하 main / training-----------------------------------

In [27]:
def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def train_model(model, train_iter, epoch):
    total_epoch_loss = 0
    total_epoch_acc = 0
    #model.cuda()
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        text = batch.text[0]
        target = batch.tml
        
        #print(target)
        target = torch.autograd.Variable(target).long()
        #print(target)
        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
        if (text.size()[0] is not 32):# One of the batch returned by BucketIterator has length different than 32.
            continue
        optim.zero_grad()
        prediction = model(text)[0]
        loss = loss_fn(prediction, target)
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

def eval_model(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text = batch.text[0]
            if (text.size()[0] is not 32):
                continue
            target = batch.tml
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            prediction = model(text)[0]
            loss = loss_fn(prediction, target)
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)


In [28]:
learning_rate = 2e-5
batch_size = 32
output_size = 2
hidden_size = 256
embedding_length = 300

In [29]:
from models.LSTM_Attn import AttentionModel

attn_model = AttentionModel(batch_size, output_size, hidden_size, vocab_size, embedding_length, word_embeddings)
loss_fn = F.cross_entropy

In [ ]:
for epoch in range(10):
    train_loss, train_acc = train_model(attn_model, train_iter, epoch)
    val_loss, val_acc = eval_model(attn_model, valid_iter)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')
    

Epoch: 1, Idx: 100, Training Loss: 0.6100, Training Accuracy:  71.88%
Epoch: 01, Train Loss: 0.654, Train Acc: 62.80%, Val. Loss: 0.650701, Val. Acc: 61.72%
Epoch: 2, Idx: 100, Training Loss: 0.7280, Training Accuracy:  59.38%
Epoch: 02, Train Loss: 0.646, Train Acc: 63.87%, Val. Loss: 0.641828, Val. Acc: 63.67%
Epoch: 3, Idx: 100, Training Loss: 0.6672, Training Accuracy:  59.38%
Epoch: 03, Train Loss: 0.637, Train Acc: 64.99%, Val. Loss: 0.640009, Val. Acc: 63.71%
Epoch: 4, Idx: 100, Training Loss: 0.6423, Training Accuracy:  65.62%


In [60]:
test_sen1="\
This apartment has: 1 double bed, 1 double sofa bed. Modern and bright, next to Park Güell This apartment will please all types of travellers, from executives to those seeking a relaxing break or some family fun. In the area of Gracia, and 10 minutes from one of Gaudi's famous masterpieces - Park Guell which has some of the most spectacular views of Barcelona with wonderful architecture, creativity and is a true escape from the hustle and bustle. The apartment is situated in a quiet residential street, but within moments of meandering streets, plazas, restaurants and bars. You are also only a few metro stops from Barcelona center \
"
test_length=len(test_sen1.split())
test_sen1 = TEXT.preprocess(test_sen1)
test_sen1 = [[TEXT.vocab.stoi[x] for x in test_sen1]]

In [61]:
from torch.autograd import Variable

In [62]:
test_sen = np.asarray(test_sen1)
test_sen = torch.LongTensor(test_sen)
test_tensor = Variable(test_sen, volatile=True)

/Users/stella-yj/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [64]:
attn_model.eval()
output, attn = attn_model(test_tensor, 1)
out = F.softmax(output, 1)
if (torch.argmax(out[0]) == 0):
    print ("score: top")
else:
    print ("score: lower")

score: top


In [65]:
test=attn.data[0][:test_length].numpy()

In [66]:
test

array([-0.08159694,  0.00458118, -0.01767293,  0.09412864, -0.11506867,
        0.01726426,  0.04962238,  0.01793724, -0.15467203,  0.03013881,
        0.09347076, -0.06337129,  0.07855573,  0.08885551, -0.02046573,
       -0.21204625,  0.14854722, -0.08275432, -0.09802559,  0.07851447,
        0.09990759,  0.04428056, -0.2040136 , -0.31677812, -0.0089146 ,
        0.12858163, -0.08899454,  0.0148458 ,  0.20416369,  0.04608628,
        0.13166374,  0.0453263 , -0.00497186, -0.12119875,  0.05600179,
       -0.05852107, -0.01309242,  0.01602482, -0.11330383, -0.00984977,
       -0.00332575,  0.16418904, -0.00249401, -0.07878935, -0.01465761,
        0.01753616, -0.07591183, -0.08512107, -0.00303029,  0.06365106,
       -0.04617692, -0.07635199,  0.07949017,  0.2204923 , -0.06026598,
       -0.03518634,  0.01809526, -0.2024592 ,  0.10682987, -0.01957087,
       -0.04169717, -0.02687893, -0.15076293, -0.11335219, -0.2073339 ,
        0.03774687, -0.02590011,  0.20387594, -0.06426351,  0.05

In [67]:
attn.size()

torch.Size([1, 256])

In [68]:
plt.matshow(test)


ValueError: not enough values to unpack (expected 2, got 1)